# Examples of scheduled circuit  (circuit with duration)

## Manual construction of T2 experiment

In [1]:
from qiskit import QuantumCircuit
qc = QuantumCircuit(1, 1, name="t2_experiment")
qc.h(0)
qc.delay(100, 0, unit='ns')
qc.h(0)
qc.measure(0, 0)
print(qc.name, qc.data)

t2_experiment [(HGate(num_qubits=1, duration=None), [Qubit(QuantumRegister(1, 'q'), 0)], []), (Delay(num_qubits=1, duration=1.0000000000000001e-07), [Qubit(QuantumRegister(1, 'q'), 0)], []), (HGate(num_qubits=1, duration=None), [Qubit(QuantumRegister(1, 'q'), 0)], []), (Measure(num_qubits=1, duration=None), [Qubit(QuantumRegister(1, 'q'), 0)], [Clbit(ClassicalRegister(1, 'c'), 0)])]


## Schedule Bell experiment with scheduling passes

In [2]:
from qiskit import transpile
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.transpiler.passes.scheduling.asap import ASAPSchedule
from qiskit.transpiler.passes.scheduling.alap import ALAPSchedule

In [3]:
from qiskit.test.mock.backends import FakeParis
backend = FakeParis()

In [4]:
qc = QuantumCircuit(2, name="bell")
qc.h(0)
qc.delay(1000, 1, unit='ns')
qc.cx(0,1)
print(qc.name, qc.data)
transpiled = transpile(qc, backend=backend, optimization_level=0, basis_gates=['u1', 'u2', 'u3', 'cx', 'delay'])
# print(transpiled.data)
dag = circuit_to_dag(transpiled)
dag_with_delays = ASAPSchedule(backend).run(dag)
scheduled = dag_to_circuit(dag_with_delays)
print(scheduled.name, scheduled.data)

bell [(HGate(num_qubits=1, duration=None), [Qubit(QuantumRegister(2, 'q'), 0)], []), (Delay(num_qubits=1, duration=1.0000000000000002e-06), [Qubit(QuantumRegister(2, 'q'), 1)], []), (CXGate(num_qubits=2, duration=None), [Qubit(QuantumRegister(2, 'q'), 0), Qubit(QuantumRegister(2, 'q'), 1)], [])]
bell_asap [(U2Gate(num_qubits=1, duration=160), [Qubit(QuantumRegister(27, 'q'), 0)], []), (Delay(num_qubits=1, duration=4340), [Qubit(QuantumRegister(27, 'q'), 0)], []), (Delay(num_qubits=1, duration=4500), [Qubit(QuantumRegister(27, 'q'), 1)], []), (CXGate(num_qubits=2, duration=1408), [Qubit(QuantumRegister(27, 'q'), 0), Qubit(QuantumRegister(27, 'q'), 1)], [])]


###  ASAP or ALAP Schedule

In [5]:
qc = QuantumCircuit(2, name="h2")
qc.h(0)
qc.x(1)
print(qc.name, qc.data)
dag = circuit_to_dag(transpile(qc, backend=backend, optimization_level=0))

h2 [(HGate(num_qubits=1, duration=None), [Qubit(QuantumRegister(2, 'q'), 0)], []), (XGate(num_qubits=1, duration=None), [Qubit(QuantumRegister(2, 'q'), 1)], [])]


In [6]:
#ASAP
dag_with_delays = ASAPSchedule(backend).run(dag)
scheduled = dag_to_circuit(dag_with_delays)
print(scheduled.name, scheduled.data)

h2_asap [(U2Gate(num_qubits=1, duration=160), [Qubit(QuantumRegister(27, 'q'), 0)], []), (Delay(num_qubits=1, duration=160), [Qubit(QuantumRegister(27, 'q'), 0)], []), (U3Gate(num_qubits=1, duration=320), [Qubit(QuantumRegister(27, 'q'), 1)], [])]


In [7]:
#ALAP
dag_with_delays = ALAPSchedule(backend).run(dag)
scheduled = dag_to_circuit(dag_with_delays)
print(scheduled.name, scheduled.data)

h2_alap [(Delay(num_qubits=1, duration=160), [Qubit(QuantumRegister(27, 'q'), 0)], []), (U2Gate(num_qubits=1, duration=160), [Qubit(QuantumRegister(27, 'q'), 0)], []), (U3Gate(num_qubits=1, duration=320), [Qubit(QuantumRegister(27, 'q'), 1)], [])]


###  Schedule passes convert the duration units into dt (values are rounded)

In [8]:
bell = QuantumCircuit(2, name="bell_with_manual_delay_ns")
bell.h(0)
bell.delay(999, 1, unit='ns')
bell.cx(0,1)
print(bell.data)
transpiled = transpile(bell, backend=backend, optimization_level=0, basis_gates=['u1', 'u2', 'u3', 'cx', 'delay'])
dag = circuit_to_dag(transpiled)
dag_with_delays = ASAPSchedule(backend).run(dag)
scheduled = dag_to_circuit(dag_with_delays)
print(scheduled.data)

[(HGate(num_qubits=1, duration=None), [Qubit(QuantumRegister(2, 'q'), 0)], []), (Delay(num_qubits=1, duration=9.99e-07), [Qubit(QuantumRegister(2, 'q'), 1)], []), (CXGate(num_qubits=2, duration=None), [Qubit(QuantumRegister(2, 'q'), 0), Qubit(QuantumRegister(2, 'q'), 1)], [])]
[(U2Gate(num_qubits=1, duration=160), [Qubit(QuantumRegister(27, 'q'), 0)], []), (Delay(num_qubits=1, duration=4336), [Qubit(QuantumRegister(27, 'q'), 0)], []), (Delay(num_qubits=1, duration=4496), [Qubit(QuantumRegister(27, 'q'), 1)], []), (CXGate(num_qubits=2, duration=1408), [Qubit(QuantumRegister(27, 'q'), 0), Qubit(QuantumRegister(27, 'q'), 1)], [])]


/Users/itoko/swd/qiskit-dev/qiskit-sdk-py/qiskit/transpiler/passes/scheduling/asap.py:51: UserWarning: Duration of delay is rounded to 4496 dt = 9.991111e-07 s from 9.990000e-07
  UserWarning)


In [9]:
bell = QuantumCircuit(2, name="bell_with_manual_delay_unitless")
bell.h(0)
bell.delay(999, 1)
bell.cx(0,1)
print(bell.data)
transpiled = transpile(bell, backend=backend, optimization_level=0, basis_gates=['u1', 'u2', 'u3', 'cx', 'delay'])
dag = circuit_to_dag(transpiled)
dag_with_delays = ASAPSchedule(backend).run(dag)
scheduled = dag_to_circuit(dag_with_delays)
print(scheduled.data)

[(HGate(num_qubits=1, duration=None), [Qubit(QuantumRegister(2, 'q'), 0)], []), (Delay(num_qubits=1, duration=999), [Qubit(QuantumRegister(2, 'q'), 1)], []), (CXGate(num_qubits=2, duration=None), [Qubit(QuantumRegister(2, 'q'), 0), Qubit(QuantumRegister(2, 'q'), 1)], [])]
[(U2Gate(num_qubits=1, duration=160), [Qubit(QuantumRegister(27, 'q'), 0)], []), (Delay(num_qubits=1, duration=839), [Qubit(QuantumRegister(27, 'q'), 0)], []), (Delay(num_qubits=1, duration=999), [Qubit(QuantumRegister(27, 'q'), 1)], []), (CXGate(num_qubits=2, duration=1408), [Qubit(QuantumRegister(27, 'q'), 0), Qubit(QuantumRegister(27, 'q'), 1)], [])]
